In [1]:
import pandas as pd
import numpy as np
import glob
import os
from openpyxl import load_workbook
from functools import reduce
pd.options.display.float_format = '{:,.2f}'.format

#### ACUMULADO

In [2]:
cumfiles = glob.glob(r'C:\Users\Cristian.Aparicio\Documents\DISPERSIONES\2023\11. NOVIEMBRE\QUERIES\*.csv', recursive=False)
cumfiles

['C:\\Users\\Cristian.Aparicio\\Documents\\DISPERSIONES\\2023\\11. NOVIEMBRE\\QUERIES\\SETTLEMENT 2023-11-01.csv',
 'C:\\Users\\Cristian.Aparicio\\Documents\\DISPERSIONES\\2023\\11. NOVIEMBRE\\QUERIES\\SETTLEMENT 2023-11-02.csv',
 'C:\\Users\\Cristian.Aparicio\\Documents\\DISPERSIONES\\2023\\11. NOVIEMBRE\\QUERIES\\SETTLEMENT 2023-11-03.csv',
 'C:\\Users\\Cristian.Aparicio\\Documents\\DISPERSIONES\\2023\\11. NOVIEMBRE\\QUERIES\\SETTLEMENT 2023-11-04.csv',
 'C:\\Users\\Cristian.Aparicio\\Documents\\DISPERSIONES\\2023\\11. NOVIEMBRE\\QUERIES\\SETTLEMENT 2023-11-05.csv',
 'C:\\Users\\Cristian.Aparicio\\Documents\\DISPERSIONES\\2023\\11. NOVIEMBRE\\QUERIES\\SETTLEMENT 2023-11-06.csv',
 'C:\\Users\\Cristian.Aparicio\\Documents\\DISPERSIONES\\2023\\11. NOVIEMBRE\\QUERIES\\SETTLEMENT 2023-11-07.csv',
 'C:\\Users\\Cristian.Aparicio\\Documents\\DISPERSIONES\\2023\\11. NOVIEMBRE\\QUERIES\\SETTLEMENT 2023-11-08.csv',
 'C:\\Users\\Cristian.Aparicio\\Documents\\DISPERSIONES\\2023\\11. NOVIEMBRE\\QU

In [3]:
cum_1 = pd.concat([pd.read_csv(f).assign(file_name=os.path.basename(f)) for f in cumfiles])
cum_1.columns = cum_1.columns.str.strip().str.lower() 
cum_1['report_date_local']= pd.to_datetime(cum_1['report_date_local']) 
cum_1.head()

,merchant_id,merchant_name,user_email,report_id,report_date_local,total_transactions,total_fees_charged,total_taxes,applied_fees_benefits,applied_taxes_benefits,...,total_released,total_retained,tipopago,total_due,type,created_date,created_time,updated_date,updated_time,file_name


In [4]:
cum_1.shape

(0, 26)

In [5]:
cum_distinct_0 = cum_1[(cum_1['total_due'] != 0)]
cum_distinct_0.head()

,merchant_id,merchant_name,user_email,report_id,report_date_local,total_transactions,total_fees_charged,total_taxes,applied_fees_benefits,applied_taxes_benefits,...,total_released,total_retained,tipopago,total_due,type,created_date,created_time,updated_date,updated_time,file_name


In [6]:
cum_distinct_0.shape

(0, 26)

#### PAGOS DE DISPERSIONES

In [7]:
pagos_dia_files_1 = glob.glob(r'C:\Users\Cristian.Aparicio\Documents\DISPERSIONES\2023\11. NOVIEMBRE\ARCHIVO DISPERSIONES\Dispersión*.xlsx', recursive=False)
pagos_dia_files_1

['C:\\Users\\Cristian.Aparicio\\Documents\\DISPERSIONES\\2023\\11. NOVIEMBRE\\ARCHIVO DISPERSIONES\\Dispersión-20231101.xlsx',
 'C:\\Users\\Cristian.Aparicio\\Documents\\DISPERSIONES\\2023\\11. NOVIEMBRE\\ARCHIVO DISPERSIONES\\Dispersión-20231102.xlsx',
 'C:\\Users\\Cristian.Aparicio\\Documents\\DISPERSIONES\\2023\\11. NOVIEMBRE\\ARCHIVO DISPERSIONES\\Dispersión-20231103.xlsx',
 'C:\\Users\\Cristian.Aparicio\\Documents\\DISPERSIONES\\2023\\11. NOVIEMBRE\\ARCHIVO DISPERSIONES\\Dispersión-20231104.xlsx',
 'C:\\Users\\Cristian.Aparicio\\Documents\\DISPERSIONES\\2023\\11. NOVIEMBRE\\ARCHIVO DISPERSIONES\\Dispersión-20231105.xlsx',
 'C:\\Users\\Cristian.Aparicio\\Documents\\DISPERSIONES\\2023\\11. NOVIEMBRE\\ARCHIVO DISPERSIONES\\Dispersión-20231106.xlsx',
 'C:\\Users\\Cristian.Aparicio\\Documents\\DISPERSIONES\\2023\\11. NOVIEMBRE\\ARCHIVO DISPERSIONES\\Dispersión-20231107.xlsx',
 'C:\\Users\\Cristian.Aparicio\\Documents\\DISPERSIONES\\2023\\11. NOVIEMBRE\\ARCHIVO DISPERSIONES\\Dispersión-

In [ ]:
pagos_1 = pd.concat([pd.read_excel(f,sheet_name='Swap del día').assign(file_name=os.path.basename(f)) for f in pagos_dia_files_1])
pagos_1['date_file'] = pagos_1["file_name"].str[-13:-5]
pagos_1['date_file']= pd.to_datetime(pagos_1['date_file'], format='%Y%m%d').dt.date
pagos_1['date_file']= pd.to_datetime(pagos_1['date_file'])
pagos_1.sort_values(by=['date_file'],ascending=True,inplace=True)
pagos_1['report_date_local']= pd.to_datetime(pagos_1['report_date_local']) 
pagos_1.head()

In [ ]:
pagos_1.shape

In [ ]:
pagos_1['report_date_local'].unique()

#### PAGOS DUPLICADOS TRADICIONAL

In [ ]:
duplicates_trad = pagos_1[pagos_1.duplicated(['report_id','date_file'] , keep=False)]
duplicates_trad['INDEX'] = duplicates_trad['report_id'] + '_' + duplicates_trad['date_file'].astype('str')
duplicates_trad.sort_values(by=['date_file','report_id'],inplace=True)
duplicates_trad.head()

In [ ]:
duplicates_trad.shape

In [ ]:
duplicates_trad['status'].unique()

In [ ]:
duplicates_trad.drop_duplicates(subset="INDEX", keep='last', inplace=True)

In [ ]:
duplicates_trad.head()

In [ ]:
duplicates_trad.shape

In [ ]:
pivot_duplicates_trad = pd.pivot_table(duplicates_trad,index=['tipo','date_file'],
                                       columns=['type'],
                                  values=['total_due']
                                  ,aggfunc = ['sum', len],margins=True)
pivot_duplicates_trad.reset_index(drop=False, inplace=True)
pivot_duplicates_trad.head(10)

In [ ]:
pivot_duplicates_trad.shape

#### FAST PAYMENTS 

In [ ]:
fp_files_1 = glob.glob(r'C:\Users\Cristian.Aparicio\Documents\FAST PAYMENTS\2023\11. NOVIEMBRE\ARCHIVO DISPERSIONES FP\Dispersión-FP*.xlsx', recursive=False)
fp_files_1

In [ ]:
pagos_2 = pd.concat([pd.read_excel(f,sheet_name='FAST PAYMENTS').assign(file_name=os.path.basename(f)) for f in fp_files_1])
pagos_2['date_file'] = pagos_2["file_name"].str[-15:-4]
pagos_2['date_file']= pd.to_datetime(pagos_2['date_file']) 
pagos_2.sort_values(by=['date_file'],ascending=True,inplace=True)
pagos_2['report_date_local']= pd.to_datetime(pagos_2['report_date_local']) 
pagos_2.head()

In [ ]:
pagos_2.shape

#### PAGOS REPETIDOS FAST PAYMENTS

In [ ]:
duplicates_fp = pagos_2[pagos_2.duplicated(['report_id','date_file'] , keep=False)]
duplicates_fp['INDEX'] = duplicates_fp['report_id'] + '_' + duplicates_fp['date_file'].astype('str')
duplicates_fp.sort_values(by=['date_file','report_id','Batch_time'],inplace=True)
duplicates_fp.head()

In [ ]:
duplicates_fp.shape

In [ ]:
duplicates_fp.drop_duplicates(subset="INDEX", keep='last', inplace=True)

In [ ]:
duplicates_fp.head()

In [ ]:
duplicates_fp.shape

In [ ]:
pivot_duplicates_fp = pd.pivot_table(duplicates_fp,index=['tipo','date_file'],
                                     columns=['type'],
                                  values=['total_due']
                                  ,aggfunc = ['sum', len],margins=True)
pivot_duplicates_fp.reset_index(drop=False, inplace=True)
pivot_duplicates_fp.head()

In [ ]:
pivot_duplicates_fp.shape

In [ ]:
# pivot_duplicates_fp.to_excel('DUPLICADOS FP JUNIO 2021.xlsx',index=True)

#### ALL PAGOS

In [ ]:
allpagos = pd.concat([pagos_1,pagos_2])
allpagos.head()

In [ ]:
allpagos.shape

In [ ]:
allpagos['tipo'].unique()

In [ ]:
group_allpagos = allpagos.groupby(['date_file']).agg({'total_due':sum,'report_id':'count'}).reset_index()
group_allpagos.head()

In [ ]:
group_allpagos.shape

#### PAGOS DEL DIA

In [ ]:
pagosdia = allpagos[(allpagos['tipo'].isin(['Pago del día']))]
pagosdia.head()

In [ ]:
pagosdia.shape

In [ ]:
group_dia = pagosdia.groupby(['date_file']).agg({'total_due':sum,'report_id':'count'}).reset_index()
group_dia.head()

In [ ]:
group_dia.shape

# PRIMER MATCH (SETTLED VS PAGOS DEL DIA)

In [ ]:
check_1 = cum_1.merge(pagosdia[['report_id','total_due','tipo','file_name']] ,on='report_id', how='left')
check_1['total_due_y'].fillna('NO ENCONTRADO')
check_1['tipo'].fillna('NO ENCONTRADO',inplace=True)
check_1['file_name_y'].fillna('NO ENCONTRADO',inplace=True)
# check_1.fillna({'total_due_y':'NO ENCONTRADO', 'tipo':'NO ENCONTRADO', 'file_name_y':'NO ENCONTRADO'}, inplace=True)
check_1.head()

In [ ]:
check_1['file_name_y'].unique()

In [ ]:
not_found_1 = check_1[((check_1['file_name_y'].isnull()) | (check_1['file_name_y'].isin(['NO ENCONTRADO'])))
                        & (check_1['total_due_x'] != 0)]
not_found_1.head()

In [ ]:
not_found_1.shape

In [ ]:
group_notfound = not_found_1.groupby(['report_date_local']).agg({'total_due_x':sum,'report_id':'count'}).reset_index()
group_notfound.rename(columns={'report_date_local':'date_file'},inplace=True)
group_notfound.head()

In [ ]:
group_notfound.shape

#### REPROGRAMACIONES

In [ ]:
repro = allpagos[(allpagos['tipo'].isin(['Reprogramación']))]
repro.head()

In [ ]:
repro.shape

In [ ]:
group_repro = repro.groupby(['date_file']).agg({'total_due':sum,'report_id':'count'}).reset_index()
group_repro.head()

In [ ]:
group_repro.shape

# SEGUNDO MATCH (IDENTIFICACION DE REPROGRAMACIONES)

In [ ]:
check_2 = not_found_1.merge(repro[['report_id','total_due','tipo','file_name']] ,on='report_id', how='left')
check_2.head()

In [ ]:
check_2.shape

In [ ]:
# writer = pd.ExcelWriter('REPROGRAMACIONES JUNIO 2021.xlsx', engine='xlsxwriter', options={'encoding':'utf-8'})
# check_2.to_excel(writer, sheet_name='DETAIL REPROGRAMACIONES',index=False)
# writer.save()
# writer.close()

# FECHA DE PAGOS (SETTLED VS PAGOS DEL DIA)

In [ ]:
fecha_pago = not_found_1.merge(repro[['report_id','date_file','tipo','status','file_name']] ,on='report_id', how='left')
fecha_pago.sort_values(by=['date_file'],ascending=True,inplace=True)
fecha_pago.head()

In [ ]:
fecha_pago.shape

In [ ]:
fecha_pago.drop_duplicates(subset ='report_id',keep='first',inplace=True) 

In [ ]:
fecha_pago.head()

In [ ]:
fecha_pago.shape

### AGRUPADO POR DÍA

In [ ]:
pivot_fecha_pago = pd.pivot_table(fecha_pago,index=['report_date_local'],
                                  values=['total_due_x'],aggfunc = ['sum', len],margins=True)
pivot_fecha_pago.head()

In [ ]:
pivot_fecha_pago.shape

# PAGOS MANUALES

In [ ]:
file_manual = glob.glob(r'G:\.shortcut-targets-by-id\1Oe28Mf9uSidMamjllDen2gkNnC59eSUr\3.-Settlement\5.-HISTORICO FILES PAGOS\01. TRADICIONAL\2023\11. NOVEMBER\MANUALES\*.xlsx', recursive=False)
file_manual

In [ ]:
manuales_all = pd.concat([pd.read_excel(f,sheet_name='Formato Pago Manual',skiprows=1
                                      ,converters={'Cta. CLABE de depósito':str,'report_id':str})
                          .assign(file_name=os.path.basename(f)) for f in file_manual])
manuales_all['Date']= pd.to_datetime(manuales_all['Date'])  
manuales_all['Tipo'] = 'MANUAL'

In [ ]:
manuales_final = manuales_all[['Date','ME ID','ME NAME',' (mxn)','Banco','Justificación para el pago','Tipo','Report ID']]
manuales_final.head()

In [ ]:
manuales_final.shape

In [ ]:
manuales_final.columns

In [ ]:
group_manual = manuales_final.groupby(['Date']).agg({' (mxn)':sum,'Report ID':'count'}).reset_index()
group_manual.rename(columns={'Date':'date_file','Report ID':'report_id'},inplace=True)
group_manual.head()

In [ ]:
group_manual.shape

# ACUMULADOS SETTLEMENT

In [ ]:
conc_disp = cum_distinct_0.groupby(['report_date_local']).agg({'total_due':sum,'report_id':'count'}).reset_index()
conc_disp.rename(columns={'report_date_local':'date_file'}, inplace=True)
conc_disp.head()

In [ ]:
conc_disp.shape

# MERGING ALL DATA FRAMES

In [ ]:
total_1 = conc_disp.merge(group_repro[['date_file','total_due','report_id']],on='date_file',how='left') \
.merge(group_manual,on='date_file',how='left')
total_1[' (mxn)'].fillna(0,inplace=True)
total_1['report_id'].fillna(0,inplace=True)
total_1.head()


In [ ]:
total_1['count_total_pago_dia'] = total_1['report_id_x'] + total_1['report_id_y'] + total_1['report_id']
total_1['sum_total_pago_dia'] = total_1['total_due_x'] + total_1['total_due_y'] + total_1[' (mxn)']


In [ ]:
total_1

# ACUMULADOS PAGO DEL DIA

In [ ]:
total_2 = group_allpagos.merge(group_notfound[['date_file','total_due_x','report_id']],on='date_file',how='left')\
.merge(group_manual,on='date_file',how='left')
total_2[' (mxn)'].fillna(0,inplace=True)
total_2['report_id'].fillna(0,inplace=True)
total_2['total_due_x'].fillna(0,inplace=True)
total_2['report_id_y'].fillna(0,inplace=True)
total_2.head()

In [ ]:
total_2['count_swap_dia'] = total_2['report_id_x'] + total_2['report_id_y'] + total_2['report_id']
total_2['sum_swap_dia'] = total_2['total_due'] + total_2['total_due_x'] + total_2[' (mxn)']

In [ ]:
total_2.head()

In [ ]:
total_2.shape

# COMPARACION FINAL

In [ ]:
final_comparacion = total_1[['date_file','count_total_pago_dia','sum_total_pago_dia']] \
.merge(total_2[['date_file','count_swap_dia','sum_swap_dia']],on='date_file',how='left')
final_comparacion.head()

In [ ]:
final_comparacion['diff_count'] = final_comparacion['count_total_pago_dia'] - final_comparacion['count_swap_dia']
final_comparacion['diff_sum'] = final_comparacion['sum_total_pago_dia'] - final_comparacion['sum_swap_dia']

In [ ]:
final_comparacion

In [ ]:
writer = pd.ExcelWriter('FASE III. SETTLED VS PAGO DIA NOVIEMBRE 2023.xlsx',
                        engine='xlsxwriter',
                        engine_kwargs={'options': {'encoding':'utf-8'}})
pivot_fecha_pago.to_excel(writer, sheet_name='ENVIADOS POSTERIOR')
fecha_pago.to_excel(writer, sheet_name='DETAIL NO ENCONTRADO',index=False)
pivot_duplicates_trad.to_excel(writer, sheet_name='PAGOS DUPLICADOS')
pivot_duplicates_fp.to_excel(writer, sheet_name='FP PAGOS DUPLICADOS')
group_manual.to_excel(writer, sheet_name='PAGOS MANUALES')
writer.close()